In [1]:
def evaluate(eva_file_path):
    data=json.loads(Path(eva_file_path).read_text())
    a=0
    for question in data:
        if question["Model Answer"] == question["Answer"]:
            a+=1
        else:
            a+=0
    accuracy=(a/len(data)*100).__round__(2)
    return accuracy

## 增加evaluation metircs

## caption metrics

Vision

In [6]:
import json
from pathlib import Path
from pprint import pprint

# ClaudeAccuracy=evaluate('/Volumes/Jennie/Reasoning/FinReasoning/output/Claude-3-5_output.json')
# ClaudeRAGAccuracy=evaluate("/Volumes/Jennie/Reasoning/FinReasoning/output/Claude-3-5_rag_output.json")
# Gemini_ProAccuracy=evaluate("/Volumes/Jennie/Reasoning/FinReasoning/output/Gemini_Pro_output.json")
# Gemini_ProRAGAccuracy=evaluate("/Volumes/Jennie/Reasoning/FinReasoning/output/Gemini_Pro_rag_output.json")
gpt_4oAccuracy=evaluate("/Volumes/Jennie/Reasoning/FinReasoning/output/gpt_4o_output.json")
gpt_4oRAGAccuracy=evaluate("/Volumes/Jennie/Reasoning/FinReasoning/output/gpt_4o_rag_output.json")


In [7]:
import pandas as pd

# 创建一个包含模型名称和准确率的字典
data = {
    'Model': ['Claude', 'ClaudeRAG', 'Gemini_Pro', 'Gemini_ProRAG', 'gpt_4o', 'gpt_4oRAG'],
    'Accuracy%': [ClaudeAccuracy, ClaudeRAGAccuracy, Gemini_ProAccuracy, Gemini_ProRAGAccuracy, gpt_4oAccuracy, gpt_4oRAGAccuracy]
}

# 创建 DataFrame
df = pd.DataFrame(data)
df

NameError: name 'ClaudeAccuracy' is not defined

In [15]:
def evaluate(data):
    a=0
    for question in data:
        if question["Model Answer"] == question["Answer"]:
            a+=1
        else:
            a+=0
    accuracy=(a/len(data)*100).__round__(2)
    return accuracy